# Автокодировщики

Объект описывается слишком большим количеством признаков? Хочется уменьшить их чилсло и не потерять важную информацию? Вам надоел PCA? Тогда автокодировщики идут к вам! 

## Почиташки 

* [Хорошая серия статей](https://habrahabr.ru/post/331382/) про автокодировщики и GAN. Немного подворовал код у них. 
* [Глава книги](http://www.deeplearningbook.org/contents/autoencoders.html) deep learning book, на которой основана серия сатей выше. 
* [Статья из блога про Keras](https://blog.keras.io/building-autoencoders-in-keras.html), на которой также основана статья выше (все воруют код у всех, просто ужас, блин). 
* [Неделя 4 курса Introduction to deep learning](https://www.coursera.org/learn/intro-to-deep-learning/home/week/4), с которой также взята часть материалов, и на которой я был ментором (возможно, что это оправдывает меня). Можно заодно [посмотреть тетрадку с этого курса.](https://github.com/hse-aml/intro-to-dl/blob/master/week4/Autoencoders-task.ipynb) Неожиданно, но она тоже с лицами...  Неожиданно, но код похож... Или даже сворован...

## Про автокодировщики 

Эпоха больших данных даёт нам слишком много данных. Часто хочется, чтобы их было немного поменьше, но при этом они выражали бы всю информацию о великом процессе порождения данных более ёмко. Не потерять в знании, но сэкономить! Именно из такого благородного помысла рождается метод главных компонент. Его посыл очень прост: давайте посмотрим на наши данные, найдём в них те направления, в которых происходит самое сильное изменение и оставим только их. При этом, в плане информации мы бы не потеряли особо много. Именно так часто поступают в случае, когда размерность данных очень большая. Находят $d$ Направлений, разброс в которых покрывает $90\%$ дисперсии, а остальное забывают. 

Например, если у нас есть выборка из карасей и щук и требуется научится отличать одних от ругих, то мы могли бы оставить только информацию о чешуйках, расположенных вдоль первой, более длинной компоненты. На качество нашего классификатора это бы повлияло слабо, но мы, при этом, избавились бы от лишней размерности, от оценки лишнего коэфициента и могли бы пустить большее число наблюдений на оценку меньшего числа параметров.  

![](https://upload.wikimedia.org/wikipedia/commons/thumb/9/90/PCA_fish.png/256px-PCA_fish.png)

Главная фишка метода главных компонент состоит в том, что он делает всю эту редукцию линейно. На самом деле все новые признаки будут представлять из себя линейные комбинации из старых. 

Идею такой редукции лишней информации можно обобщить до нелинейного случая и сделать прыжок к нейросетям, которые называются **автоэнокдерами.**  Автоэнкодеры учатся оставлять минимум информации так, чтобы по этому минимуму было возможно восстановить с очень высокой точностью исходную информацию. 

Автоэнкодеры — это нейронные сети прямого распространения, которые восстанавливают входной сигнал на выходе. Внутри у них имеется скрытый слой, который представляет собой код, описывающий модель. Автоэнкодеры конструируются таким образом, чтобы не иметь возможность точно скопировать вход на выходе. Обычно их ограничивают в размерности кода. 

![](https://hsto.org/web/cf6/228/613/cf6228613fdc4f8fb819cbd41bb677eb.png) 

Закодированную часть Входной сигнал восстанавливается с ошибками из-за потерь при кодировании, но, чтобы их минимизировать, сеть вынуждена учиться отбирать наиболее важные признаки.

Новый пример! Пусть у нас есть куча рукописных цифр и мы хотели бы научить компьютер понимать где какая цифра нарисована. Каждая цифра это картинка из пикселей размер $28 \times 28$. Всего у нас $28^2$ наблюдаемые переменные. Это слишком много! Явно не каждый пиксель несёт в себе информацию о том какая цифра нарисована на картинке. Большая часть пикселей бесполезна. 

Чтобы отобрать самые важные нелинейные комбинации из пикселей, мы можем сделать следующее: 

![](https://blog.keras.io/img/ae/autoencoder_schema.jpg)

Нейросеть получает на вход цифру, пытается оставить 10 самых важных нейлинейных комбинаций из пикселей, а после по этим самым важным комбинациям, она пытается восстановить цифру назад с как можно большей точностью. Ограничение на то сколько самых важных нелинейных пикселей должно остаться, заставляет нашу сетку стараться отобрать исключительно всё самое важное, что можно найти на картинке. Метод главных компонент будет частным, самым простым случаем, такой нейронной сетки. 

Первая часть автокодировщика $g(x)$ называется **encoder**. Она пытается закодировать картинку. Вторая часть $f(h)$ называет **decoder**. Она пытается раскодировать картинку. Автоэккодер, изменяя $f$ и $g$ пытается выучить тождественную функцию 

$$x = f(g(x)),$$ 

минимизируя какой-то функционал ошибки 

$$L(x, f(g(x))).$$ 

При этом выучить тождественную выборку он не может, так как в сердцевине находится  **бутылочное горлышко** с довольно маленьким числом нейронов. В итоге сетка вынуждена отаравлять в сердцевину только те комбинации фичей, по которым легче всего восстановить входные данные, и таким образом отсеивает ненужную информацию. В горлышке в результате обучения оказываются наши новые нелинейные фичи. 

In [6]:
import tensorflow as tf 
tf.__version__

'2.0.0'

In [7]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers as L

In [8]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

## 1. Данные.

Будем учить свой автокодировщик для человеческих лиц. Пользоваться будем lfw-датасетом:

* http://vis-www.cs.umass.edu/lfw

In [ ]:
from sklearn.datasets import fetch_lfw_people

lfw_people = fetch_lfw_people(min_faces_per_person=0, resize=0.5, color=False,
                              slice_=(slice(94, 190, None), slice(78, 174, None)))

In [ ]:
lfw_people.data.dtype

In [ ]:
lfw_people.images.shape

In [ ]:
lfw_people.images[0].shape

In [ ]:
lfw_people.target_names[0] # имена людей

In [ ]:
from sklearn.model_selection import train_test_split

X = lfw_people.images
IMG_SHAPE = X.shape[1:]

X = X.astype('float32') / 255.0
X = X[...,None]
X_train, X_test = train_test_split(X, test_size=0.1, random_state=42)

In [ ]:
X.shape

In [ ]:
plt.imshow(X_train[0,:,:,0], cmap='gray')
X_train[0].shape

Хочу ещё данных!

In [ ]:
cols = 8
rows = 2
fig = plt.figure(figsize=(2 * cols - 1, 2.5 * rows - 1))
for i in range(cols):
    for j in range(rows):
        random_index = np.random.randint(0, len(X_train))
        ax = fig.add_subplot(rows, cols, i * rows + j + 1)
        ax.grid(False)
        ax.axis('off')
        ax.imshow(X_train[random_index,:,:,0], cmap='gray')
plt.show()

С этим можно уже учить модели. 

## 2. Собираем автокодировщик для PCA 

Как мы уже выяснили, простейшим автокодировщиком является PCA. Один слой на входе, бутылочное горлышко с новыми фичами, один слой на выходе и PCA готов. 

In [ ]:
input_shape = X_train.shape[1:]
input_shape

In [ ]:
np.prod(input_shape) # пикселей

In [ ]:
def build_pca_autoencoder(img_shape, code_size=36):
    # code_size это размерность кодированного представления (фичей на выходе)
    
    # Энкодер 
    
    #############
    # Ваш код 
    #############
    
    # Декодер 
    
    #############
    # Ваш код 
    #############
    
    return encoder,decoder

Осталось только собрать модель из моделей. 

In [ ]:
# Сконструировали энкодер и декодер 
encoder, decoder = build_pca_autoencoder(input_shape, code_size=36)

inp = L.Input(input_shape)  # Вход
code = encoder(inp)         # Энкодер от входа 
reconstruction = decoder(code)  # Декодер от кода 

# Скрепили две наши сетки в одну общую модель
autoencoder = Model(inp, reconstruction)

# Собираем сеть с методом оптимизации и выбранной ошибкой 
autoencoder.compile('adam', 'mse') 

# Посмотрим на параметры 
autoencoder.summary()

Обратите внимание что энкодер восстанавливает вход на выходе. У нас нет меток для обучения, а надо, чтобы ими были сами по себе исходные картинки. Немного подправим генератор данных, чтобы это правда было так. 

In [ ]:
# Обучаем модель 
hist_1 = autoencoder.fit(x = X_train, y = X_train,  # Обратите внимание что энкодер восстанавливает вход на выходе 
                epochs = 40,                        # Никакие y тут не нужны. Это просто лэйблы... 
                batch_size = 256,
                shuffle = True,
                validation_data = [X_test, X_test])

Наш автокодировщик обучился. это довольно приятная новость. Попробуем отрисовать оригинальные изображения, их компактные представления и то как по этим представлениям изображения восстанавливаются. 

In [ ]:
# Функция для отрисовки ооигинальных цифр, их представления и восстановленного рисунка
def visualize(img, encoder, decoder):
    code = encoder.predict(img[None])[0]
    reco = decoder.predict(code[None])[0]

    plt.subplot(1,3,1)
    plt.title("Original")
    plt.imshow(img[:,:,0], cmap='gray')

    plt.subplot(1,3,2)
    plt.title("Code")
    plt.imshow(code.reshape([code.shape[-1]//2,-1]))

    plt.subplot(1,3,3)
    plt.title("Reconstructed")
    plt.imshow(reco.clip(0,1)[:,:,0], cmap='gray')
    plt.show()

In [ ]:
score = autoencoder.evaluate(X_test, X_test,verbose=0)
print("Final MSE:",score)

for i in range(3):
    img = X_test[i]
    visualize(img,encoder,decoder)

Теперь по оставшимся 36 фичам (вместо оригинальных 784) можно строить классификаторы. Попробуйте на досуге заняться этим весёлым занятием, а мы движемся дальше. 

## 3 Глубокий автоэнкодер 

Всегда можно лучше! Сконструируем глубокий автокодировщик для вычленения нелинейных фичей. 

![ ](https://pbs.twimg.com/media/CYggEo-VAAACg_n.png:small)

* Обратите внимание, что при постройке автокодировщика не должно быть слоёв меньших, чем бутылочное горлышко (выход энкодера). 
* Также обратите внимание, что вполне уместно использовать свёртки и пулинг, но такой автокодировщик будет построен чуточку ниже. 

In [ ]:
def build_deep_autoencoder(input_shape, code_size=36):
    
    #############
    # Ваш код 
    #############
    
    return encoder,decoder

# Сконструировали энкодер и декодер 
encoder,decoder = build_deep_autoencoder(input_shape, code_size=36)

In [ ]:
# Объединили всё вместе и задали процедуру оптимизации для сетки

    #############
    # Ваш код 
    #############

In [ ]:
# Обучаем модель 
hist_2 = autoencoder.fit(x = X_train, y = X_train,  # Обратите внимание что энкодер восстанавливает вход на выходе 
                epochs = 40,                        # Никакие y тут не нужны. Это просто лэйблы... 
                batch_size = 256,
                shuffle = True,
                validation_data=[X_test, X_test])

In [ ]:
score = autoencoder.evaluate(X_test, X_test,verbose=0)
print("Final MSE:",score)

for i in range(3):
    img = X_test[i]
    visualize(img,encoder,decoder)

## 4. Свёрточный автокодировщик 

Добавим свёрточные слои и немного макспулинга.  Часть, которая относится к энкодеру будет стандартной. Это просто свёрточные слои и пулинг. Кончается она обычным свёрточным слоем с `code_size` нейронов. В качестве функции активации берите `elu`. 

Попробуйте повторить свёртку и пулинг $2\times2$ четыре раза с ядром размера $3 \times 3$ и `padding='same'`. У слоёв поставьте число каналов: $32$, $64$, $128$, $256$.


Для декодера мы будем использовать 

`L.Conv2DTranspose(filters=?, kernel_size=(3, 3), strides=2, activation='elu', padding='same')`

и

`UpSampling2D((2,2))`. 

Сначала попробуйте скомбинировать эти два варианта, а после вспомните объяснение про `Conv2DTranspose` с предыдущей пары :) 

In [9]:
def build_conv_autoencoder(input_shape, code_size=36):
    
    #############
    # Ваш код 
    #############
   
    
    return encoder,decoder

In [ ]:
# Ваш код, чтобы скрепит всё вместе

    #############
    # Ваш код 
    #############

In [ ]:
# sgd = tf.keras.optimizers.Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
autoencoder.compile(optimizer='adam', loss='mse') # Собираем сеть с методом оптимизации и выбранной ошибкой 

# Посмотрим на параметры 
autoencoder.summary()

In [ ]:
# Модель будет обучаться ну оооочень долго... 
# Ошибка падает довольно быстро, добавлять свёртку было хорошей идеей
hist_3 = autoencoder.fit(X_train, X_train,
                        epochs=10,
                        batch_size=256,
                        shuffle=True,
                        validation_data=(X_test, X_test))

In [ ]:
score = autoencoder.evaluate(X_test, X_test,verbose=0)
print("Final MSE:",score)

for i in range(3):
    img = X_test[i]
    visualize(img,encoder,decoder)

## 5. Denoising автоэнкодер 

Автоэнкодеры можно обучить убирать шум из данных: для этого надо на вход подавать зашумленные данные и на выходе сравнивать с данными без шума. Создадим модель, которая будет зашумлять входное изображение, а после пытаться избавить его от шума. 

In [ ]:
# Добавляем на картинку немного шума 
def apply_gaussian_noise(X,sigma=0.01):
    return X + sigma*np.random.normal(loc=0.0, scale=1.0, size=X.shape) 

In [ ]:
plt.subplot(1,4,1)
plt.imshow(X_train[0].reshape(48, 48), cmap = 'gray')
plt.subplot(1,4,2)
plt.imshow(apply_gaussian_noise(X_train[:1],sigma=0.01)[0].reshape(48, 48), cmap = 'gray')
plt.subplot(1,4,3)
plt.imshow(apply_gaussian_noise(X_train[:1],sigma=0.1)[0].reshape(48, 48), cmap = 'gray')
plt.subplot(1,4,4)
plt.imshow(apply_gaussian_noise(X_train[:1],sigma=0.5)[0].reshape(48, 48), cmap = 'gray');

In [ ]:
# Зашумляем данные 
X_train_noisy = apply_gaussian_noise(X_train, sigma=0.1)
X_test_noisy = apply_gaussian_noise(X_test, sigma=0.1)

# Убираем все заскоки за отрезок [0;1]
X_train_noisy = np.clip(X_train_noisy, 0., 1.)
X_test_noisy = np.clip(X_test_noisy, 0., 1.)

Собираем сеточку из предыдущего пункта.

In [ ]:
# размер горлышка имеет смысл взять побольше (почему?)
encoder_noisy, decoder_noisy = build_deep_autoencoder(input_shape, code_size=1024)

inp = L.Input(input_shape)
code = encoder_noisy(inp)
reconstruction = decoder_noisy(code)

autoencoder_noisy = Model(inp, reconstruction)
autoencoder_noisy.compile('adam', 'mse')

autoencoder_noisy.summary()

In [ ]:
hist_noise = autoencoder_noisy.fit(X_train_noisy, X_train,
                        epochs=10,
                        batch_size=256,
                        shuffle=True,
                        validation_data=(X_test_noisy, X_test))

In [ ]:
score = autoencoder_noisy.evaluate(X_test_noisy, X_test,verbose=0)
print("Final MSE:",score)

for i in range(3):
    img = X_test_noisy[i]
    visualize(img,encoder,decoder)

## 6.  Поиск похожих изображений 

С помощью найденного горлышка можно искать похожие друг на друга изображения. Чтобы ускорить процесс поиска, мы будем использовать похожий на метод ближайших соседей, но более быстрый алгоритм под названием [Locality Sensitive Hashing forest.](http://scikit-learn.org/0.16/modules/generated/sklearn.neighbors.LSHForest.html)

In [ ]:
images = X_train
codes = encoder.predict(images)

# хорошая идея ставить по ходу архитекрутуры проверки на размерности :) 
assert len(codes) == len(images)

In [ ]:
import sklearn
sklearn.__version__

In [ ]:
from sklearn.neighbors import LSHForest
lshf = LSHForest(n_estimators=50).fit(codes)

In [ ]:
def get_similar(image, n_neighbors=5):
    
    code = encoder.predict(image[None])
    
    (distances,),(idx,) = lshf.kneighbors(code,n_neighbors=n_neighbors)
    
    return distances,images[idx]

In [ ]:
def show_similar(image):
    
    distances,neighbors = get_similar(image,n_neighbors=11)
    
    plt.figure(figsize=[8,6])
    plt.subplot(3,4,1)
    plt.imshow(image.reshape(48,48), cmap = 'gray')
    plt.title("Original image")
    
    for i in range(11):
        plt.subplot(3,4,i+2)
        plt.imshow(neighbors[i].reshape(48,48), cmap = 'gray')
        plt.title("Dist=%.3f"%distances[i])
    plt.show()

In [ ]:
show_similar(X_test[2])

In [ ]:
show_similar(X_test[500])

In [ ]:
show_similar(X_test[66])

## 7.  Прогулка по пространству эмбедингов.

Можно брать два эмбединга, искать между ними линейную комбинацию и пытаться прогуляться от одного к другому. Попробуйте вытащить из для двух заданных изображений `image1` и `image2` несколько линейных комбинаций эмбедингов и посмотреть как выглядит картинка с плавным переходом от одной картинки к другой. 

In [ ]:
path_len = 10

for _ in range(5):
    image1,image2 = X_test[np.random.randint(0,len(X_test),size=2)]

    code1, code2 = # ваш код для поиска эмбединга

    plt.figure(figsize=[15,6])
    for i,a in enumerate(np.linspace(0,1,num=path_len)):

        # ваш код для расчёта эмбединга
        # и его перевода в картинку

        plt.subplot(1, path_len, i+1)
        plt.imshow(output_image.reshape(48,48), cmap = 'gray')
        plt.title("a=%.2f"%a)        
    plt.show()